# 財報選股

參考資料：

* [Python 財報爬蟲](https://goo.gl/rVZhZa)
* [請問有高手會匯入這個網頁的資料到excel嗎? ](https://www.mobile01.com/topicdetail.php?f=291&t=3360713)

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
def financial_statement(year, season, exchange='sii', type='綜合損益表'):
    if year > 1911:
        year -= 1911
        
    if type == '綜合損益表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '營益分析查詢彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        print('type does not match')

    # 一些參數：
    # TYPEK => 市場別
    # sii>上市
    # otc>上櫃
    # rotc>興櫃
    # pub>公開發行

    # year => 年度
    # season => 季別
    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK': exchange,
        'year': year,
        'season': season,
    })
    
    r.encoding = 'utf8'
    
    dfs = pd.read_html(r.text)
    
    
    for i, df in enumerate(dfs):
        df.columns = df.iloc[0]
        dfs[i] = df.iloc[1:]
        
    df = pd.concat(dfs).applymap(lambda x: x if x != '--' else np.nan)
    df = df[df['公司代號'] != '公司代號']
    df = df[~df['公司代號'].isnull()]
    if any(df.columns.str.contains("合計：")):
        df = df.loc[:, ~df.columns.str.contains("合計：")]
    return df

In [3]:
df = financial_statement(2017, 2, type='營益分析查詢彙總表')

C:\Users\Malo\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [4]:
df[df['公司名稱']=='台泥']

公司代號 公司名稱 毛利率(%)(營業毛利)/(營業收入) 營業利益率(%)(營業利益)/(營業收入) 營業收入(百萬元)  \
1  1101   台泥               18.01                 13.08  45893.85   

  稅前純益率(%)(稅前純益)/(營業收入) 稅後純益率(%)(稅後純益)/(營業收入)  
1                 12.54                  9.23

In [5]:
cond1 = df['毛利率(%)(營業毛利)/(營業收入)'].astype(float) > 20
cond2 = df['營業利益率(%)(營業利益)/(營業收入)'].astype(float) > 5

In [6]:
df[cond1 & cond2]

公司代號     公司名稱 毛利率(%)(營業毛利)/(營業收入) 營業利益率(%)(營業利益)/(營業收入)  營業收入(百萬元)  \
9    1203     味王公司               31.09                 15.37    3076.03   
13   1216       統一               33.30                  6.08  196218.29   
20   1227       佳格               27.73                  9.41   11996.01   
21   1229     聯華實業               22.24                 10.61    2356.89   
24   1233       天仁               57.26                  9.86    1058.28   
28   1256  鮮活果汁-KY               30.13                 15.41    1422.17   
29   1262    綠悅-KY               29.53                 23.65    3751.43   
35   1307     三芳化工               32.56                 16.02    5057.42   
44   1319       東陽               25.22                 11.21   11962.03   
46   1323       永裕               23.30                 12.10    1517.63   
52   1338    廣華-KY               27.04                 12.24    3668.62   
53   1339       昭輝               31.84                 18.95    1363.10   
54   1340    勝悅-KY               31.96                 25.91    1507.94   
55   1402      遠東新               23.09                  6.39  103290.00   
56   1410       南染               37.89                 22.99     273.27   
59   1416       廣豐               26.80                  7.73    2041.70   
68   1436      華友聯               23.71                 14.49     981.14   
69   1437     勤益投控               71.00                 55.26     299.31   
74   1442       名軒               26.28                 18.39    1875.50   
91   1463     強盛染整               50.42                 40.90    1101.12   
98   1470     大統新創               22.42                  6.46     366.87   
104  1476       儒鴻               26.70                 15.88   11089.23   
106  1503       士電               20.35                  7.41   10668.20   
107  1504     東元電機               25.12                  7.64   24989.43   
109  1507     永大機電               28.18                 11.05    8769.34   
119  1522      堤維西               22.80                  5.10    8092.80   
120  1524     耿鼎企業               20.26                 10.10    1292.91   
122  1526       日馳               27.57                 15.40     558.56   
123  1527       鑽全               29.06                 20.86    1936.17   
127  1531      高林股               26.87                 11.89    1116.54   
..    ...      ...                 ...                   ...        ...   
882  8427    基勝-KY               24.98                  7.96    2965.80   
883  8429    金麗-KY               27.36                 18.79    2399.77   
888  8464       億豐               48.75                 25.80    9603.79   
889  8466   美吉吉-KY               26.60                 16.51    1838.36   
891  8473      山林水               32.46                 27.21    1266.77   
892  8478     東哥遊艇               30.50                 10.41    1771.67   
893  8480    泰昇-KY               31.03                 16.74     726.14   
894  8481       政伸               44.33                 21.94     518.05   
898  8926      台汽電               25.51                  6.05     499.44   
900  8996       高力               26.96                  7.70     878.77   
901  9802    鈺齊-KY               22.39                 13.27    4919.43   
903  9904     寶成工業               25.89                  6.14  136821.82   
905  9906      欣巴巴               22.80                  7.26     359.16   
907  9908      大台北               20.76                 13.75    1906.98   
908  9910     豐泰企業               21.51                  8.90   27663.08   
909  9911     台灣櫻花               34.59                 12.35    2586.66   
913  9917       中保               36.10                 16.53    6398.95   
914  9918      欣天然               26.82                 14.32    1120.49   
917  9924     台灣福興               20.85                 10.48    3906.96   
918  9925     新光保全               33.34                 11.47    3568.58   
919  9926     新海瓦斯               23.90                 16.79    1126.63   
926  9934     成霖企業            